Задача регрессии (для IC50)

In [58]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score #будем использовать кросс-валидацию, чтобы получить наиболее релевантные метрики

model_linear = LinearRegression()

model_linear.fit(X_train_ic, y_train_ic)
pred_linear_ic = model_linear.predict(X_test_ic)

scores_linear = cross_val_score(model_linear, X_train_ic, y_train_ic, cv=5, scoring='r2')
print("Среднее R2:", scores_linear.mean())

Среднее R2: 0.25480398195720105


In [59]:
from sklearn.metrics import r2_score
r2_test = r2_score(y_test_ic, pred_linear_ic)
print("R2 на тесте:", r2_test)

R2 на тесте: 0.3403781793341438


In [60]:
from sklearn.ensemble import RandomForestRegressor

model_forest = RandomForestRegressor()

model_forest.fit(X_train_ic, y_train_ic)
pred_forest_ic = model_forest.predict(X_test_ic)

scores_forest = cross_val_score(model_forest, X_train_ic, y_train_ic, cv=5, scoring='r2')
print("Среднее R2:", scores_forest.mean())

Среднее R2: 0.4370137227044751


In [61]:
r2_test_for = r2_score(y_test_ic, pred_forest_ic)
print("R2 на тесте:", r2_test_for)

R2 на тесте: 0.39951600346802496


In [62]:
from sklearn.ensemble import GradientBoostingRegressor

model_grad = GradientBoostingRegressor()

model_grad.fit(X_train_ic, y_train_ic)
pred_grad_ic = model_grad.predict(X_test_ic)

scores_grad = cross_val_score(model_grad, X_train_ic, y_train_ic, cv=5, scoring='r2')
print("Среднее R2:", scores_grad.mean())

Среднее R2: 0.44055403013113936


In [63]:
r2_test_grad = r2_score(y_test_ic, pred_grad_ic)
print("R2 на тесте:", r2_test_grad)

R2 на тесте: 0.4397512398595178


Применение только алгоритмов машинного обучения даже при должной обработке и кросс-валидации дают крайне низкие результаты. В силу специфики поставленной задачи, возможно, работать со всеми химическими соединениями и биологическими классами подряд не совсем логично - они могут быть различными по типу соединения и молекулярной структуре, из-за чего работа с видами этих соединений по отдельности (применение к каждому виду алгоритмов машинного обучения) покажет лучший результат. Для решения задачи можно попробовать применить кластеризацию, и только после этого использовать алгоритмы машинного обучения, так как кластеризация способна выявить эти взаимосвязи.

In [65]:
from sklearn.cluster import KMeans

kmeans_ic = KMeans(n_clusters=3, random_state=42)
kmeans_ic.fit(X_train_ic)

train_clusters = kmeans_ic.predict(X_train_ic)
test_clusters = kmeans_ic.predict(X_test_ic)

C:\Users\dfu\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


Чтобы посчитать метрики на каждом кластере, будем использовать функцию. Также внутри этой функции будем делить загруженные в функции выборки снова на train и test, чтобы предотвратить завышение метрик.

In [67]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_clusters(X, y, clusters, set_name):
    results = []
    for cluster_id in np.unique(clusters):
        mask = clusters == cluster_id  #используем маску, чтобы обеспечить соответствие кластеров в X и y выборках
        X_cluster = X[mask]
        y_cluster = y[mask]
        
        if len(X_cluster) > 10: 
            X_train_cl, X_val_cl, y_train_cl, y_val_cl = train_test_split(
                X_cluster, y_cluster, test_size=0.3, random_state=42
            )
            model = LinearRegression().fit(X_train_cl, y_train_cl)
            y_pred = model.predict(X_val_cl)
            
            metrics = {
                'Выборка': set_name,
                'Кластер': cluster_id,
                'Размер Train в функции': len(y_train_cl),
                'Размер Val в функции': len(y_val_cl),
                'MAE': mean_absolute_error(y_val_cl, y_pred),
                'MSE': mean_squared_error(y_val_cl, y_pred),
                'R2': r2_score(y_val_cl, y_pred)
            }
            results.append(metrics)
    
    return pd.DataFrame(results)

train_metrics = evaluate_clusters(X_train_ic, y_train_ic, train_clusters, "Train_ic")
test_metrics = evaluate_clusters(X_test_ic, y_test_ic, test_clusters, "Test_ic")

all_metrics = pd.concat([train_metrics, test_metrics], ignore_index=True)

print("\nМетрики по кластерам:")
print(all_metrics.to_markdown(index=False, floatfmt=".3f"))


Метрики по кластерам:
| Выборка   |   Кластер |   Размер Train в функции |   Размер Val в функции |   MAE |    MSE |      R2 |
|:----------|----------:|-------------------------:|-----------------------:|------:|-------:|--------:|
| Train_ic  |         0 |                      109 |                     47 | 0.635 |  0.721 |  -0.695 |
| Train_ic  |         1 |                      192 |                     83 | 0.689 |  0.891 |  -0.185 |
| Train_ic  |         2 |                      188 |                     81 | 0.725 |  0.768 |   0.500 |
| Test_ic   |         0 |                       42 |                     18 | 2.918 | 21.759 | -74.195 |
| Test_ic   |         1 |                       67 |                     29 | 1.721 |  4.823 |  -4.857 |
| Test_ic   |         2 |                      101 |                     44 | 1.215 |  2.645 |  -1.199 |


In [68]:
def evaluate_clusters_for(X, y, clusters, set_name):
    results = []
    for cluster_id in np.unique(clusters):
        mask = clusters == cluster_id
        X_cluster = X[mask]
        y_cluster = y[mask]
        
        if len(X_cluster) > 10:  
            X_train_cl, X_val_cl, y_train_cl, y_val_cl = train_test_split(
                X_cluster, y_cluster, test_size=0.3, random_state=42
            )
            model = GradientBoostingRegressor().fit(X_train_cl, y_train_cl)
            y_pred = model.predict(X_val_cl)
            
            metrics = {
                'Выборка': set_name,
                'Кластер': cluster_id,
                'Размер Train в функции': len(y_train_cl),
                'Размер Val в функции': len(y_val_cl),
                'MAE': mean_absolute_error(y_val_cl, y_pred),
                'MSE': mean_squared_error(y_val_cl, y_pred),
                'R2': r2_score(y_val_cl, y_pred)
            }
            results.append(metrics)
    
    return pd.DataFrame(results)

train_metrics_for = evaluate_clusters_for(X_train_ic, y_train_ic, train_clusters, "Train_ic")
test_metrics_for = evaluate_clusters_for(X_test_ic, y_test_ic, test_clusters, "Test_ic")

all_metrics_for = pd.concat([train_metrics_for, test_metrics_for], ignore_index=True)

print("\nМетрики по кластерам:")
print(all_metrics_for.to_markdown(index=False, floatfmt=".3f"))


Метрики по кластерам:
| Выборка   |   Кластер |   Размер Train в функции |   Размер Val в функции |   MAE |   MSE |     R2 |
|:----------|----------:|-------------------------:|-----------------------:|------:|------:|-------:|
| Train_ic  |         0 |                      109 |                     47 | 0.477 | 0.335 |  0.212 |
| Train_ic  |         1 |                      192 |                     83 | 0.602 | 0.596 |  0.207 |
| Train_ic  |         2 |                      188 |                     81 | 0.634 | 0.578 |  0.624 |
| Test_ic   |         0 |                       42 |                     18 | 0.680 | 0.611 | -1.111 |
| Test_ic   |         1 |                       67 |                     29 | 0.706 | 0.851 | -0.033 |
| Test_ic   |         2 |                      101 |                     44 | 0.720 | 1.058 |  0.121 |


In [69]:
def evaluate_clusters_grad(X, y, clusters, set_name):
    results = []
    for cluster_id in np.unique(clusters):
        mask = clusters == cluster_id
        X_cluster = X[mask]
        y_cluster = y[mask]
        
        if len(X_cluster) > 10:  
            X_train_cl, X_val_cl, y_train_cl, y_val_cl = train_test_split(
                X_cluster, y_cluster, test_size=0.3, random_state=42
            )
            model = RandomForestRegressor().fit(X_train_cl, y_train_cl)
            y_pred = model.predict(X_val_cl)
            
            metrics = {
                'Выборка': set_name,
                'Кластер': cluster_id,
                'Размер Train в функции': len(y_train_cl),
                'Размер Val в функции': len(y_val_cl),
                'MAE': mean_absolute_error(y_val_cl, y_pred),
                'MSE': mean_squared_error(y_val_cl, y_pred),
                'R2': r2_score(y_val_cl, y_pred)
            }
            results.append(metrics)
    
    return pd.DataFrame(results)

train_metrics_grad = evaluate_clusters_grad(X_train_ic, y_train_ic, train_clusters, "Train_ic")
test_metrics_grad = evaluate_clusters_grad(X_test_ic, y_test_ic, test_clusters, "Test_ic")

all_metrics_grad = pd.concat([train_metrics_grad, test_metrics_grad], ignore_index=True)

print("\nМетрики по кластерам:")
print(all_metrics_grad.to_markdown(index=False, floatfmt=".3f"))


Метрики по кластерам:
| Выборка   |   Кластер |   Размер Train в функции |   Размер Val в функции |   MAE |   MSE |     R2 |
|:----------|----------:|-------------------------:|-----------------------:|------:|------:|-------:|
| Train_ic  |         0 |                      109 |                     47 | 0.448 | 0.295 |  0.306 |
| Train_ic  |         1 |                      192 |                     83 | 0.579 | 0.554 |  0.263 |
| Train_ic  |         2 |                      188 |                     81 | 0.600 | 0.509 |  0.669 |
| Test_ic   |         0 |                       42 |                     18 | 0.567 | 0.484 | -0.671 |
| Test_ic   |         1 |                       67 |                     29 | 0.713 | 0.806 |  0.022 |
| Test_ic   |         2 |                      101 |                     44 | 0.701 | 0.922 |  0.234 |


In [70]:
kmeans_cc = KMeans(n_clusters=3, random_state=42)
kmeans_cc.fit(X_train_cc)

train_clusters_cc = kmeans_cc.predict(X_train_cc)
test_clusters_cc = kmeans_cc.predict(X_test_cc)

C:\Users\dfu\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [71]:
train_metrics_grad_cc = evaluate_clusters_grad(X_train_cc, y_train_cc, train_clusters_cc, "Train_ic")
test_metrics_grad_cc = evaluate_clusters_grad(X_test_cc, y_test_cc, test_clusters_cc, "Test_ic")

all_metrics_grad_cc = pd.concat([train_metrics_grad_cc, test_metrics_grad_cc], ignore_index=True)

print("\nМетрики по кластерам:")
print(all_metrics_grad_cc.to_markdown(index=False, floatfmt=".3f"))


Метрики по кластерам:
| Выборка   |   Кластер |   Размер Train в функции |   Размер Val в функции |   MAE |   MSE |     R2 |
|:----------|----------:|-------------------------:|-----------------------:|------:|------:|-------:|
| Train_ic  |         0 |                      102 |                     44 | 0.473 | 0.391 |  0.170 |
| Train_ic  |         1 |                      355 |                    153 | 0.443 | 0.356 |  0.405 |
| Train_ic  |         2 |                       32 |                     14 | 0.694 | 0.697 |  0.187 |
| Test_ic   |         0 |                       35 |                     16 | 0.501 | 0.351 | -0.374 |
| Test_ic   |         1 |                      159 |                     69 | 0.438 | 0.301 |  0.081 |
| Test_ic   |         2 |                       15 |                      7 | 0.514 | 0.374 |  0.316 |


In [72]:
kmeans_si = KMeans(n_clusters=3, random_state=42)
kmeans_si.fit(X_train_si)

train_clusters_si = kmeans_si.predict(X_train_si)
test_clusters_si = kmeans_si.predict(X_test_si)

C:\Users\dfu\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [73]:
train_metrics_grad_si = evaluate_clusters_grad(X_train_si, y_train_si, train_clusters_si, "Train_ic")
test_metrics_grad_si = evaluate_clusters_grad(X_test_si, y_test_si, test_clusters_si, "Test_ic")

all_metrics_grad_si = pd.concat([train_metrics_grad_si, test_metrics_grad_si], ignore_index=True)

print("\nМетрики по кластерам:")
print(all_metrics_grad_si.to_markdown(index=False, floatfmt=".3f"))


Метрики по кластерам:
| Выборка   |   Кластер |   Размер Train в функции |   Размер Val в функции |   MAE |   MSE |     R2 |
|:----------|----------:|-------------------------:|-----------------------:|------:|------:|-------:|
| Train_ic  |         0 |                       86 |                     37 | 0.536 | 0.399 |  0.234 |
| Train_ic  |         1 |                      207 |                     90 | 0.546 | 0.495 |  0.079 |
| Train_ic  |         2 |                      196 |                     84 | 0.554 | 0.453 |  0.105 |
| Test_ic   |         0 |                       37 |                     16 | 0.486 | 0.387 | -0.363 |
| Test_ic   |         1 |                      100 |                     44 | 0.605 | 0.632 | -0.125 |
| Test_ic   |         2 |                       72 |                     32 | 0.559 | 0.512 |  0.060 |


Использование различных алгоритмов машинного обучения с учителем в сочетании с кластеризацией не дало нужных результатов - R2 ниже нуля, только местами метрики становятся относительно приемлемыми. Данное решение может быть приемлемым, если нужно в процессе исследования данных выделить некотрые наиболее важные физико-химические параметры отдельных кластеров и вывести их с помощью feature_importance. Однако из-за общих плохих оценок, можно сделать вывод, что кластеризация в поставленных задачах не поможет улучшить результат. Следовательно, выберем наиболее эффективный алгоритм для каждой отдельной задачи регрессии без кластеризации. В задаче для IC50 лучше всего показал себя алгоритм градиентного бустинга. Подберем для него лучшие гиперпараметры с помощью RandomizedSearch. Попробуем различные алгоритмы на других задачах регрессии.

In [75]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),  # случайное число от 0.01 до 0.31
}

random_search = RandomizedSearchCV(
    estimator=GradientBoostingRegressor(),
    param_distributions=param_dist,
    n_iter=50,       # сколько комбинаций попробовать
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train_ic, y_train_ic)

print("Лучшие параметры:", random_search.best_params_)

Лучшие параметры: {'learning_rate': 0.11753971856328177, 'max_depth': 3, 'n_estimators': 77}


In [76]:
grad_for_ic = GradientBoostingRegressor(
    n_estimators = random_search.best_params_['n_estimators'],
    max_depth = random_search.best_params_['max_depth'],
    learning_rate = random_search.best_params_['learning_rate'],
    random_state=42
)
grad_for_ic.fit(X_train_ic, y_train_ic)

GradientBoostingRegressor(learning_rate=0.11753971856328177, n_estimators=77,
                          random_state=42)

In [77]:
grad_boost_ic = grad_for_ic.predict(X_test_ic)

In [78]:
r2_test_grad_for_ic = r2_score(y_test_ic, grad_boost_ic )
print("R2 на тесте:", r2_test_grad_for_ic)

R2 на тесте: 0.4276572363509815


Метрика получилась довольно неплохой. Будем применять сочетание RandomizedSearch вместе с наиболее подходящими алгоритмами классического машинного обучения для решения других задач исследования.